In [3]:
import numpy as np
import pandas as pd
from scipy import stats
import os

from common import OUTPUTPATH
from models import linear_pcc
import data

In [4]:
# estimators here
STATE = np.random.RandomState(seed=1000)
linear_pcc.fit(data.x1, data.y)

In [5]:
num_importance = 100 # defult values show all variables
from sklearn.inspection import permutation_importance

def individual_importance_dataframe(est, X, y, scoring='neg_log_loss', num_importance=100000, n_repeats=30):
    """
    This algorithm use linear estimators and get the importance variables where p < 0.05
    To easy comparison, we set importance score: 1- p
    """
    r = permutation_importance(est, X, y, n_repeats=n_repeats, scoring=scoring)
    non_zero_indx = np.where(r.importances_mean > 0)
    indx = r.importances_mean[non_zero_indx].argsort()[::-1]
    important_variables = X.columns[non_zero_indx][indx]
    importance_scores = r.importances_mean[non_zero_indx][indx]
    coef = est.coef_[0][non_zero_indx][indx]
    df = pd.DataFrame({'variables': important_variables[:num_importance], 'coef': coef[:num_importance], 'importance': importance_scores[:num_importance]})
    return df

def linear_importance_dataframe(est, X, y, scoring = 'neg_log_loss', num_importance=100000, n_repeats=30):
    """
    Combine all morphologies plots together
    """
    data1 = pd.concat([X, y], axis=1)
    cnt = None
    for i in range(3):
        col_indx = -4 + i

        estimator, X, y = est[i], data1.iloc[:, :col_indx], data.y.iloc[:, i]
        temp_df = individual_importance_dataframe(est=estimator, X=X, y=y, scoring=scoring, num_importance=num_importance, n_repeats=n_repeats)
        if not cnt: 
            df = temp_df
            cnt = 1
        else:
            df = pd.concat([df, temp_df], axis=1)

    upper_columns = ['Sphere', 'Worm', 'Vesicle']
    lower_columns = ['variables', 'coef', 'importance']
    df.columns = pd.MultiIndex.from_product([upper_columns, lower_columns], names=['Phase', 'Property'])
    return df

show all importance dataframe for Sphere, Worm, Vesicle

In [6]:
linear_importance_dataframe(linear_pcc.fitted_, data.abbrev_x1, data.y, num_importance=100000)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted w

Phase         Sphere                                Worm            \
Property   variables      coef    importance   variables      coef   
0         mw_tot_cre  0.000738  7.980805e+00  mw_tot_cre -0.000420   
1         mv_tot_cre -0.046841  3.783763e+00  mv_tot_cre  0.024149   
2             dp_cre -0.054974  3.760046e+00      dp_cre  0.026126   
3           apol_cna  0.231271  1.052951e+00  mw_tot_cna -0.000266   
4            psa_cna -0.043151  3.281867e-01    apol_cna -0.065693   
5         mv_tot_cna  0.044544  2.380768e-01  mv_tot_cna  0.025160   
6             dp_cna  0.044611  2.025216e-01        conc  0.075894   
7            psa_cre -0.169942  1.204407e-01     psa_cna  0.010289   
8               conc -0.107200  6.043887e-02     psa_cre  0.061203   
9         mw_tot_cna -0.000139  4.058973e-02      mw_cna  0.004199   
10            mw_cre  0.020345  8.319404e-03      mw_cre -0.025906   
11            mw_cna -0.001091  1.143751e-03        temp  0.019900   
12         clogp_cna  0.022651  8.386795e-04      dp_cna  0.003485   
13              temp -0.005243  3.895534e-04      sphere -0.017840   
14           charged  0.019577  2.693296e-04   clogp_cna -0.010471   
15         clogp_cre  0.012587  9.956426e-05    apol_cre  0.003100   
16        mass_ratio  0.008407  4.954180e-05     charged -0.006160   
17         vol_ratio  0.008301  4.865721e-05   clogp_cre -0.004785   
18                ph  0.000607  5.800077e-06      mv_cna  0.000993   
19          apol_cre  0.000081  1.143904e-06  mass_ratio -0.001771   
20              salt -0.000030  1.609043e-08   vol_ratio -0.001642   
21               NaN       NaN           NaN          ph -0.000919   
22               NaN       NaN           NaN        salt -0.000418   
23               NaN       NaN           NaN      mv_cre  0.000150   
24               NaN       NaN           NaN         NaN       NaN   

Phase                      Vesicle                          
Property    importance   variables      coef    importance  
0         6.162994e+00  mw_tot_cre -0.000204  2.397071e+00  
1         1.550817e+00  mv_tot_cre  0.013208  1.116525e+00  
2         1.390675e+00      dp_cre  0.015380  1.063644e+00  
3         1.645097e-01  mw_tot_cna -0.000157  6.239715e-02  
4         1.423314e-01     psa_cna  0.009357  2.956265e-02  
5         8.463098e-02      mw_cna -0.003235  1.557988e-02  
6         4.805958e-02    apol_cna -0.012553  9.531407e-03  
7         3.302468e-02        temp  0.013764  7.668041e-03  
8         2.716029e-02        conc  0.011315  4.278364e-03  
9         2.570507e-02  mv_tot_cna  0.004351  4.225794e-03  
10        7.403866e-03     psa_cre  0.006827  1.621519e-03  
11        5.941467e-03      sphere -0.003967  3.776521e-04  
12        1.626221e-03      dp_cna -0.002485  2.986822e-04  
13        7.837410e-04      mw_cre -0.006919  2.677311e-04  
14        4.723402e-04          ph -0.002722  1.499752e-04  
15        1.270316e-04   clogp_cna -0.000956  3.440335e-05  
16        9.203443e-05   clogp_cre -0.001084  2.419326e-05  
17        4.418490e-05    apol_cre  0.000384  1.846264e-05  
18        6.690469e-06        worm -0.000525  7.435417e-06  
19        6.550988e-06  mass_ratio -0.000468  5.181399e-06  
20        5.572708e-06   vol_ratio -0.000449  4.743624e-06  
21        1.016446e-06     charged -0.000407  3.636385e-06  
22        4.783966e-07      mv_cna  0.000116  4.457302e-07  
23        2.805372e-07      mv_cre  0.000025  6.480962e-08  
24                 NaN        salt -0.000051  6.361488e-08

In [8]:
top5_linear_dataframe = linear_importance_dataframe(linear_pcc.fitted_, data.abbrev_x1, data.y, num_importance=5)
top5_linear_dataframe.to_csv(os.path.join(OUTPUTPATH, 'Top5_linear_dataframe.csv'))
top5_linear_dataframe


/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LogisticRegressionCV was fitted w

Phase         Sphere                             Worm                       \
Property   variables      coef importance   variables      coef importance   
0         mw_tot_cre  0.000738   8.007907  mw_tot_cre -0.000420   6.121813   
1         mv_tot_cre -0.046841   3.777106  mv_tot_cre  0.024149   1.568045   
2             dp_cre -0.054974   3.748520      dp_cre  0.026126   1.410971   
3           apol_cna  0.231271   1.060032  mw_tot_cna -0.000266   0.170789   
4            psa_cna -0.043151   0.334054    apol_cna -0.065693   0.149355   

Phase        Vesicle                       
Property   variables      coef importance  
0         mw_tot_cre -0.000204   2.373600  
1         mv_tot_cre  0.013208   1.140040  
2             dp_cre  0.015380   1.081454  
3         mw_tot_cna -0.000157   0.061926  
4            psa_cna  0.009357   0.030507